In [1]:
import argparse
import yaml
import os
import logging
import shutil
import numpy as np
from PIL import Image 
logger = logging.getLogger()
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
logger.addHandler(handler)

import torch
import torch.optim as optim
import torchvision.transforms as transforms
from diffusers import DDIMScheduler, EulerDiscreteScheduler
from datasets import load_dataset
from diffusers.utils.torch_utils import randn_tensor
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

from main.wmdiffusionxl import WMDetectStableDiffusionXLPipeline
from main.wmpatch import GTWatermark, GTWatermarkMulti
from main.utils import *
from loss.loss import LossProvider
from loss.pytorch_ssim import ssim

/opt/conda/envs/unmarked/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Necessary Setup for All Sections

In [2]:
logging.info(f'===== Load Config =====')
device = torch.device('cuda')
with open('./example/config/config.yaml', 'r') as file:
    cfgs = yaml.safe_load(file)
logging.info(cfgs)

===== Load Config =====
{'method': 'ZoDiac', 'save_img': './example/output/', 'model_id': 'stabilityai/stable-diffusion-2-1-base', 'gen_seed': 0, 'empty_prompt': True, 'w_type': 'single', 'w_channel': 3, 'w_radius': 10, 'w_seed': 10, 'start_latents': 'init_w', 'iters': 100, 'save_iters': [100], 'loss_weights': [10.0, 0.1, 1.0, 0.0], 'ssim_threshold': 0.92, 'detect_threshold': 0.9}


In [3]:
logging.info(f'===== Init Pipeline =====')
if cfgs['w_type'] == 'single':
    wm_pipe = GTWatermark(device, w_channel=cfgs['w_channel'], w_radius=cfgs['w_radius'], generator=torch.Generator(device).manual_seed(cfgs['w_seed']))
elif cfgs['w_type'] == 'multi':
    wm_pipe = GTWatermarkMulti(device, w_settings=cfgs['w_settings'], generator=torch.Generator(device).manual_seed(cfgs['w_seed']))

# scheduler = DDIMScheduler.from_pretrained(cfgs['model_id'], subfolder="scheduler")
# pipe = WMDetectStableDiffusionPipeline.from_pretrained(cfgs['model_id'], scheduler=scheduler).to(device)
# pipe.set_progress_bar_config(disable=True)

===== Init Pipeline =====


In [4]:
base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
checkpoint = "sdxl_lightning_4step_unet.safetensors"
device = 'cuda'

pipe = WMDetectStableDiffusionXLPipeline.from_pretrained(
    base,
    torch_dtype=torch.float16,
    variant="fp16"
).to(device)
scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config,
    timestep_spacing="trailing",
    prediction_type="epsilon"
)
# scheduler = DDIMScheduler.from_pretrained(pipe.scheduler.config)
pipe.scheduler = scheduler
pipe.unet.load_state_dict(
    load_file(
        hf_hub_download(repo, checkpoint),
        device=device
    )
)
pipe.set_progress_bar_config(disable=True)

/opt/conda/envs/unmarked/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  7.03it/s]
/opt/conda/envs/unmarked/lib/python3.12/site-packages/diffusers/configuration_utils.py:134: FutureWarning: Accessing config attribute `force_zeros_for_empty_prompt` directly via 'WMDetectStableDiffusionXLPipeline' object attribute is deprecated. Please access 'force_zeros_for_empty_prompt' over 'WMDetectStableDiffusionXLPipeline's config object instead, e.g. 'scheduler.config.force_zeros_for_empty_prompt'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


In [5]:
imagename = 'pepper.tiff'
gt_img_tensor = get_img_tensor(f'./example/input/{imagename}', device)
wm_path = cfgs['save_img']

## Image Watermarking

In [6]:
# Step 1: Get init noise
def get_init_latent(img_tensor, pipe, text_embeddings, guidance_scale=1.0):
    # DDIM inversion from the given image
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        img_latents = pipe.get_image_latents(img_tensor, sample=False)
        reversed_latents = pipe.forward_diffusion(
            latents=img_latents,
            text_embeddings=text_embeddings,
            guidance_scale=guidance_scale,
            num_inference_steps=4,
        )
        return reversed_latents

empty_text_embeddings = pipe.get_text_embedding('')
init_latents_approx = get_init_latent(gt_img_tensor, pipe, empty_text_embeddings)

RuntimeError: The size of tensor a (2048) must match the size of tensor b (1024) at non-singleton dimension 1

In [6]:
# Step 2: prepare training
init_latents = init_latents_approx.detach().clone()
init_latents.requires_grad = True
optimizer = optim.Adam([init_latents], lr=0.01)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.3) 

totalLoss = LossProvider(cfgs['loss_weights'], device)
loss_lst = [] 

/opt/conda/envs/unmarked/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/unmarked/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:01<00:00, 535MB/s] 
/home/ec2-user/ZoDiac/loss/loss.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle mo

In [ ]:
# Step 3: train the init latents
for i in range(cfgs['iters']):
    logging.info(f'iter {i}:')
    init_latents_wm = wm_pipe.inject_watermark(init_latents)
    if cfgs['empty_prompt']:
        pred_img_tensor = pipe('', guidance_scale=1.0, num_inference_steps=50, output_type='tensor', use_trainable_latents=True, init_latents=init_latents_wm).images
    else:
        pred_img_tensor = pipe(prompt, num_inference_steps=50, output_type='tensor', use_trainable_latents=True, init_latents=init_latents_wm).images
    loss = totalLoss(pred_img_tensor, gt_img_tensor, init_latents_wm, wm_pipe)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    loss_lst.append(loss.item())
    # save watermarked image
    if (i+1) in cfgs['save_iters']:
        path = os.path.join(wm_path, f"{imagename.split('.')[0]}_{i+1}.png")
        save_img(path, pred_img_tensor, pipe)
torch.cuda.empty_cache()

iter 0:
/home/ec2-user/ZoDiac/main/wmdiffusion.py:198: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  num_channels_latents = self.unet.in_channels
/opt/conda/envs/unmarked/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Watermark 0.0000, Image 0.5736, Perp 0.9297, SSIM 0.5659 Total Loss 2.0692
iter 1:
Watermark 0.0000, Image 0.4470, Perp 0.8598, SSIM 0.5189 Total Loss 1.8257
iter 2:
Watermark

## Postprocessing with Adaptive Enhancement

In [ ]:
# hyperparameter
ssim_threshold = cfgs['ssim_threshold']

In [ ]:
wm_img_path = os.path.join(wm_path, f"{imagename.split('.')[0]}_{cfgs['save_iters'][-1]}.png")
wm_img_tensor = get_img_tensor(wm_img_path, device)
ssim_value = ssim(wm_img_tensor, gt_img_tensor).item()
logging.info(f'Original SSIM {ssim_value}')

In [ ]:
def binary_search_theta(threshold, lower=0., upper=1., precision=1e-6, max_iter=1000):
    for i in range(max_iter):
        mid_theta = (lower + upper) / 2
        img_tensor = (gt_img_tensor-wm_img_tensor)*mid_theta+wm_img_tensor
        ssim_value = ssim(img_tensor, gt_img_tensor).item()

        if ssim_value <= threshold:
            lower = mid_theta
        else:
            upper = mid_theta
        if upper - lower < precision:
            break
    return lower

optimal_theta = binary_search_theta(ssim_threshold, precision=0.01)
logging.info(f'Optimal Theta {optimal_theta}')

img_tensor = (gt_img_tensor-wm_img_tensor)*optimal_theta+wm_img_tensor

ssim_value = ssim(img_tensor, gt_img_tensor).item()
psnr_value = compute_psnr(img_tensor, gt_img_tensor)

tester_prompt = '' 
text_embeddings = pipe.get_text_embedding(tester_prompt)
det_prob = 1 - watermark_prob(img_tensor, pipe, wm_pipe, text_embeddings)

path = os.path.join(wm_path, f"{os.path.basename(wm_img_path).split('.')[0]}_SSIM{ssim_threshold}.png")
save_img(path, img_tensor, pipe)
logging.info(f'SSIM {ssim_value}, PSNR, {psnr_value}, Detect Prob: {det_prob} after postprocessing')

## Attack Watermarked Image with Individual Attacks

In [ ]:
from main.wmattacker import *
from main.attdiffusion import ReSDPipeline

logging.info(f'===== Init Attackers =====')
att_pipe = ReSDPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, revision="fp16")
att_pipe.set_progress_bar_config(disable=True)
att_pipe.to(device)

attackers = {
    'diff_attacker_60': DiffWMAttacker(att_pipe, batch_size=5, noise_step=60, captions={}),
    'cheng2020-anchor_3': VAEWMAttacker('cheng2020-anchor', quality=3, metric='mse', device=device),
    'bmshj2018-factorized_3': VAEWMAttacker('bmshj2018-factorized', quality=3, metric='mse', device=device),
    'jpeg_attacker_50': JPEGAttacker(quality=50),
    'rotate_90': RotateAttacker(degree=90),
    'brightness_0.5': BrightnessAttacker(brightness=0.5),
    'contrast_0.5': ContrastAttacker(contrast=0.5),
    'Gaussian_noise': GaussianNoiseAttacker(std=0.05),
    'Gaussian_blur': GaussianBlurAttacker(kernel_size=5, sigma=1),
    'bm3d': BM3DAttacker(),
}

In [ ]:
logging.info(f'===== Start Attacking... =====')

post_img = os.path.join(wm_path, f"{imagename.split('.')[0]}_{cfgs['save_iters'][-1]}_SSIM{ssim_threshold}.png")
for attacker_name, attacker in attackers.items():
    print(f'Attacking with {attacker_name}')
    os.makedirs(os.path.join(wm_path, attacker_name), exist_ok=True)
    att_img_path = os.path.join(wm_path, attacker_name, os.path.basename(post_img))
    attackers[attacker_name].attack([post_img], [att_img_path])

## Attack Watermarked Image with Combined Attacks

In [ ]:
from main.wmattacker import *
from main.attdiffusion import ReSDPipeline

case_list = ['w/ rot', 'w/o rot']

logging.info(f'===== Init Attackers =====')
att_pipe = ReSDPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, revision="fp16")
att_pipe.set_progress_bar_config(disable=True)
att_pipe.to(device)

In [ ]:
post_img = os.path.join(wm_path, f"{imagename.split('.')[0]}_{cfgs['save_iters'][-1]}_SSIM{ssim_threshold}.png")

for case in case_list:
    print(f'Case: {case}')
    if case == 'w/ rot':
        attackers = {
        'diff_attacker_60': DiffWMAttacker(att_pipe, batch_size=5, noise_step=60, captions={}),
        'cheng2020-anchor_3': VAEWMAttacker('cheng2020-anchor', quality=3, metric='mse', device=device),
        'bmshj2018-factorized_3': VAEWMAttacker('bmshj2018-factorized', quality=3, metric='mse', device=device),
        'jpeg_attacker_50': JPEGAttacker(quality=50),
        'rotate_90': RotateAttacker(degree=90),
        'brightness_0.5': BrightnessAttacker(brightness=0.5),
        'contrast_0.5': ContrastAttacker(contrast=0.5),
        'Gaussian_noise': GaussianNoiseAttacker(std=0.05),
        'Gaussian_blur': GaussianBlurAttacker(kernel_size=5, sigma=1),
        'bm3d': BM3DAttacker(),
        }
        multi_name = 'all'
    elif case == 'w/o rot':
        attackers = {
        'diff_attacker_60': DiffWMAttacker(att_pipe, batch_size=5, noise_step=60, captions={}),
        'cheng2020-anchor_3': VAEWMAttacker('cheng2020-anchor', quality=3, metric='mse', device=device),
        'bmshj2018-factorized_3': VAEWMAttacker('bmshj2018-factorized', quality=3, metric='mse', device=device),
        'jpeg_attacker_50': JPEGAttacker(quality=50),
        'brightness_0.5': BrightnessAttacker(brightness=0.5),
        'contrast_0.5': ContrastAttacker(contrast=0.5),
        'Gaussian_noise': GaussianNoiseAttacker(std=0.05),
        'Gaussian_blur': GaussianBlurAttacker(kernel_size=5, sigma=1),
        'bm3d': BM3DAttacker(),
        }
        multi_name = 'all_norot'
        
    
    os.makedirs(os.path.join(wm_path, multi_name), exist_ok=True)
    att_img_path = os.path.join(wm_path, multi_name, os.path.basename(post_img))
    for i, (attacker_name, attacker) in enumerate(attackers.items()):
        print(f'Attacking with {attacker_name}')
        if i == 0:
            attackers[attacker_name].attack([post_img], [att_img_path], multi=True)
        else:
            attackers[attacker_name].attack([att_img_path], [att_img_path], multi=True)

## Detect Watermark

In [ ]:
post_img = os.path.join(wm_path, f"{imagename.split('.')[0]}_{cfgs['save_iters'][-1]}_SSIM{ssim_threshold}.png")

attackers = ['diff_attacker_60', 'cheng2020-anchor_3', 'bmshj2018-factorized_3', 'jpeg_attacker_50', 
             'brightness_0.5', 'contrast_0.5', 'Gaussian_noise', 'Gaussian_blur', 'rotate_90', 'bm3d', 
             'all', 'all_norot']

tester_prompt = '' # assume at the detection time, the original prompt is unknown
text_embeddings = pipe.get_text_embedding(tester_prompt)

In [ ]:
logging.info(f'===== Testing the Watermarked Images {post_img} =====')
det_prob = 1 - watermark_prob(post_img, pipe, wm_pipe, text_embeddings)
logging.info(f'Watermark Presence Prob.: {det_prob}')

In [ ]:
logging.info(f'===== Testing the Attacked Watermarked Images =====')
for attacker_name in attackers:
    if not os.path.exists(os.path.join(wm_path, attacker_name)):
        logging.info(f'Attacked images under {attacker_name} not exist.')
        continue
        
    logging.info(f'=== Attacker Name: {attacker_name} ===')
    det_prob = 1 - watermark_prob(os.path.join(wm_path, attacker_name, os.path.basename(post_img)), pipe, wm_pipe, text_embeddings)
    logging.info(f'Watermark Presence Prob.: {det_prob}')